In [18]:
using HTTP
using Dates
using CSV
using DataFrames
using Geodesy
using JSON

In [2]:
#=
    DarkSky API has 2 features: Forecast and Timemachine.  Timemachine allows you to obtain historical
    weather data.
    format
        forecast_url = https://api.darksky.net/forecast/[key]/[latitude],[longitude]

        timemachine_url = https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]
            time is unix timestamp. 

=#
 #=
    For historical data, the unix_timestamp would have to be an array spaning the time frame of the desired
    historical data.

    Data has to be saved into a CSV format
=#
#= 
    In order to work with the earthquake dates which are given in UTC in the following form
    2018-12-29T03:39:09.740Z
    The "Z" indicating Zulu? or UTC time has to be stripped from the string.  A complex example would be as follows:
    round(Int, Dates.datetime2unix(DateTime("2018-12-29T03:39:09.740Z")))
    which should give the result 
    1546054750
    which roughly translates to Dec 29, 2018 at 3:39am UTC
=#
#=
    The object provided by the weather api for historical data, includes sunrise and sunset time and perhaps temperature 
    either at sunrise/sunset or an hour that is close to that time which would be called the terminator. Time.  The first
    call to the api would give us the sunrise/sunset and that could be used to get the temperature for sunrise/sunset
    for the previous days as well as previous years
=#

In [3]:
earthquake_datetime = "2000-01-08T16:47:20.580Z"
base_url = "https://api.darksky.net/forecast"
filename = "/home/antonio/Repos/iono2/earthquakes_csv/1999_2017_eq79.csv"

"/home/antonio/Repos/iono2/earthquakes_csv/1999_2017_eq79.csv"

In [4]:
df = DataFrame(CSV.File(filename))

,Column1,datetime,latitude,longitude,depth,magnitude
,Int64,String,Float64,Float64,Float64,Float64
1,96,2000-01-08T16:47:20.580Z,-16.925,-174.248,183.4,7.2
2,571,2000-02-25T01:43:58.640Z,-19.528,173.818,33.0,7.1
3,891,2000-03-28T11:00:22.510Z,22.338,143.73,126.5,7.6
4,1149,2000-04-23T09:27:23.320Z,-28.307,-62.99,608.5,7.0
5,1297,2000-05-04T04:21:16.210Z,-1.105,123.573,26.0,7.6
6,1420,2000-05-12T18:43:18.120Z,-23.548,-66.452,225.0,7.2
7,1674,2000-06-04T16:28:26.170Z,-4.721,102.087,33.0,7.9
8,1997,2000-06-18T14:44:13.310Z,-13.802,97.453,10.0,7.9
9,2869,2000-08-06T07:27:12.900Z,28.856,139.556,394.8,7.4


In [5]:
df_eq = df[["datetime", "latitude", "longitude"]]

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[5]:1
└ @ Core In[5]:1


,datetime,latitude,longitude
,String,Float64,Float64
1,2000-01-08T16:47:20.580Z,-16.925,-174.248
2,2000-02-25T01:43:58.640Z,-19.528,173.818
3,2000-03-28T11:00:22.510Z,22.338,143.73
4,2000-04-23T09:27:23.320Z,-28.307,-62.99
5,2000-05-04T04:21:16.210Z,-1.105,123.573
6,2000-05-12T18:43:18.120Z,-23.548,-66.452
7,2000-06-04T16:28:26.170Z,-4.721,102.087
8,2000-06-18T14:44:13.310Z,-13.802,97.453
9,2000-08-06T07:27:12.900Z,28.856,139.556


In [6]:
function read_apikey(apikey_file)
    key = open(apikey_file) do file
          readlines(file)
        end 
    return String(key[1])
end

read_apikey (generic function with 1 method)

In [7]:
apikey = read_apikey("/home/antonio/Repos/iono2/julia_scripts/darksky_api.txt")
println("Done reading apikey!")

Done reading apikey!


In [8]:
function unix_time(earthquake_time)
    eq_time = strip(earthquake_time, last(earthquake_time))
    return round(Int, Dates.datetime2unix(DateTime(eq_time)))
end

unix_time (generic function with 1 method)

In [9]:
function darksky_api_call(url)
    try
        response = HTTP.get(url)
        return String(response.body)
        catch e
        return "Error occured: $e"
    end
end

darksky_api_call (generic function with 1 method)

In [ ]:
function cities_data()
    #open file, load to a datatable,
    #use function to find the nearest point and use that lat and lon for city data
   #= if I want to use the julia script to find the closest city, 
    the large calculation is usually the firstone, because the longitude search 
    will span only within the closest longitude.  DO NOT DELETE THE SCRIPT, IT WILL BE 
    NEEDED WHEN WE CALCULATE THE CLOSEST VALUE FOR TEC AND OLR DATA.

In [ ]:
# This will return the nearest city lat, lon and the distance in kilometers
function find_nearest_city(lat, lon)
    city_df = cities_data() # This will receive a table, with the city data
    earthquake_coords = LLA(lat, lon)
    return distance(cities_coords, earthquake_coords) / 1000 # 1 KM
end

In [10]:
function historical_weather(latitude, longitude, earthquake_datetime)
    println("Looking up info for Latitude $latitude, longitud $longitude and earthquake_time $earthquake_datetime\n")
    unix_timestamp = unix_time(earthquake_datetime)
    historical_temperature_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitude) * "," * string(unix_timestamp)
    println("unix_time stamp : $unix_timestamp", "\n$historical_temperature_url")
    return darksky_api_call(historical_temperature_url)
end

historical_weather (generic function with 1 method)

In [11]:
function forecast_weather(latitude, longitude)
    forecast_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitude)
    return darksky_api_call(forecast_url)
end

forecast_weather (generic function with 1 method)

In [20]:
latitude = -18.6496
longitude = -173.9833
unix_timestamp = 947350040
1594314751
historical_temperature_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitude) * "," * string(unix_timestamp)
darksky_api_call(historical_temperature_url)


"{\"latitude\":-18.6496,\"longitude\":-173.9833,\"timezone\":\"Pacific/Tongatapu\",\"currently\":{\"time\":947350040,\"uvIndex\":0},\"hourly\":{\"data\":[{\"time\":947354400,\"temperature\":80.46,\"apparentTemperature\":86.83,\"dewPoint\":76.44,\"humidity\":0.88,\"pressure\":1006.6,\"windSpeed\":9.18,\"windBearing\":270,\"cloudCover\":0.38,\"uvIndex\":0},{\"time\":947365200,\"temperature\":84.63,\"apparentTemperature\":93.11,\"dewPoint\":75.19,\"humidity\":0.73,\"pressure\":1006.9,\"windSpeed\":11.39,\"windBearing\":270,\"cloudCover\":0.25,\"uvIndex\":6},{\"time\":947386800,\"temperature\":86.04,\"apparentTemperature\":99.19,\"dewPoint\":78.6,\"humidity\":0.79,\"pressure\":1005.6,\"windSpeed\":14.99,\"windBearing\":281,\"cloudCover\":0.63,\"uvIndex\":4}]},\"flags\":{\"sources\":[\"cmc\",\"gfs\",\"icon\",\"isd\",\"madis\"],\"nearest-station\":4.643,\"units\":\"us\"},\"offset\":14}\n"

In [14]:
historical = historical_weather(latitude, longitude, timestamp)

Looking up info for Latitude -16.925, longitud -174.258 and earthquake_time 947350040



MethodError: MethodError: no method matching strip(::Int64, ::Int64)
Closest candidates are:
  strip(::Any, !Matched::AbstractString) at strings/util.jl:222
  strip(::Any, !Matched::CategoricalValue{String,R} where R<:Integer) at deprecated.jl:65

In [12]:
#forecast_urls
forecast = forecast_weather(latitude, longitude)
f = JSON.parse(forecast)

UndefVarError: UndefVarError: latitude not defined

In [ ]:
if typeof(f["currently"]["apparentTemperature"]) == Float64
    println("Found")
    else
    println("No weather data available")

end

In [ ]:
haskey(f["currently"], "apparentTemperature")

In [ ]:
unix_timestamp = unix_time("2018-12-29T03:39:09.740Z")

In [ ]:
earthquake_time

In [ ]:
historical_temperature_url

In [ ]:
for i in eachrow(df_eq)
    earthquake_datetime = i["datetime"]
    latitude = i["latitude"]
    longitude = i["longitude"]
    
    historical_weather(latitude, longitude, earthquake_datetime)
    h = JSON.parse(historical)
    
    if haskey(h["currently"], "apparentTemperature")
        println(earthquake_datetime, " historical data available")
    else
        println(earthquake_datetime, " No Data!")
    end
    
end